In [ ]:
# ==============================================================================
# Projeto: Predição de Doenças Cardíacas (Pipeline Completo com ML)
# Nome e RM: [Fabrício Henrique Pereira, RM 563237]
# Descrição: Implementação de um pipeline de classificação supervisionada
# ==============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import time
from typing import Tuple, Dict, Any, List

# Bibliotecas de Scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, ConfusionMatrixDisplay, classification_report
)
from sklearn.exceptions import NotFittedError

# ==============================================================================
# CONFIGURAÇÕES GERAIS E LOGGING
# ==============================================================================
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)
# Configura o sistema de log para registrar as etapas importantes
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

# Constantes para garantir a reprodutibilidade
RANDOM_STATE = 42
TEST_SIZE = 0.3
np.random.seed(RANDOM_STATE)

# Dados do Dataset
URL_DATASET = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
COLUMN_NAMES = [
    'age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
    'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'
]


# ==============================================================================
# FUNÇÕES AUXILIARES DE AVALIAÇÃO
# ==============================================================================

def avaliar_modelo(y_true: pd.Series, y_pred: np.ndarray) -> Dict[str, float]:
    """Calcula e retorna as métricas de classificação chave."""
    return {
        'Acurácia': accuracy_score(y_true, y_pred),
        'Precisão': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, zero_division=0)
    }

def plotar_matriz_confusao(y_true: pd.Series, y_pred: np.ndarray, nome: str, cmap: str = 'Blues'):
    """Plota a Matriz de Confusão para visualização no Notebook."""
    cm = confusion_matrix(y_true, y_pred)
    disp = ConfusionMatrixDisplay(cm, display_labels=['Sem Doença (0)', 'Com Doença (1)'])
    fig, ax = plt.subplots(figsize=(4, 4))
    disp.plot(cmap=cmap, ax=ax)
    plt.title(f'Matriz de Confusão - {nome}')
    plt.show()


# ==============================================================================
# FUNÇÕES PRINCIPAIS DO PIPELINE
# ==============================================================================

def preparar_dados(url: str, names: List[str], test_size: float, rs: int) -> Tuple[ColumnTransformer, pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """Carrega, limpa e divide o dataset. Retorna o pré-processador e os dados."""
    logging.info("--- ETAPA 1: Carregamento e Pré-processamento ---")

    # 1. Carregamento e Limpeza Inicial
    df = pd.read_csv(url, header=None, names=names)
    logging.info(f"Dados brutos carregados: {len(df)} registros.")
    
    # Tratamento de valores desconhecidos e ausentes
    df.replace('?', np.nan, inplace=True)
    df.dropna(inplace=True)
    
    # Conversão de tipos e Definição do Alvo Binário
    df['ca'] = pd.to_numeric(df['ca'])
    df['thal'] = pd.to_numeric(df['thal'])
    df['target'] = df['target'].apply(lambda x: 1 if x > 0 else 0)

    logging.info(f"Dados limpos: {len(df)} registros.")
    logging.info(f"Distribuição da classe alvo:\n{df['target'].value_counts(normalize=True)}")

    # 2. Divisão e Feature Engineering
    X = df.drop('target', axis=1)
    y = df['target']

    # Divisão com estratificação para manter a proporção da classe alvo
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=rs, stratify=y
    )

    # Definição das colunas para o ColumnTransformer
    num_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
    cat_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

    # ColumnTransformer: Aplica transformações distintas para diferentes tipos de variáveis
    preprocessor = ColumnTransformer([
        # Padronização para modelos sensíveis à escala (LogReg, KNN)
        ('scaler', StandardScaler(), num_features),
        # Codificação One-Hot para variáveis categóricas
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features)
    ], remainder='passthrough')
    
    logging.info("Pré-processador (ColumnTransformer) definido com sucesso.")
    return preprocessor, X_train, X_test, y_train, y_test


def treinar_modelos_base(preprocessor: ColumnTransformer, X_train: pd.DataFrame, X_test: pd.DataFrame, y_train: pd.Series, y_test: pd.Series):
    """Treina os 3 modelos base e identifica o vencedor."""
    logging.info("\n--- ETAPA 2: Treinamento e Avaliação dos Modelos Base (3) ---")

    modelos = {
        "Regressão Logística": LogisticRegression(random_state=RANDOM_STATE, max_iter=1000),
        "KNN": KNeighborsClassifier(n_neighbors=7),
        "Random Forest": RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100)
    }

    resultados = {}
    metricas = pd.DataFrame(columns=['Acurácia', 'Precisão', 'Recall', 'F1-Score', 'Tempo (s)'])

    for nome, modelo in modelos.items():
        inicio = time.time()
        # Pipeline: Combina o pré-processamento com o classificador
        pipe = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', modelo)])
        
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        fim = time.time()

        # Avaliação
        m = avaliar_modelo(y_test, y_pred)
        m['Tempo (s)'] = fim - inicio
        metricas.loc[nome] = m
        resultados[nome] = pipe

        logging.info(f"  {nome} treinado | F1: {m['F1-Score']:.4f} | Tempo: {m['Tempo (s)']:.2f}s")
        plotar_matriz_confusao(y_test, y_pred, nome) # Plota para visualização no Notebook

    metricas = metricas.sort_values(by='F1-Score', ascending=False)
    melhor_nome = metricas.index[0]
    melhor_modelo = resultados[melhor_nome]

    print("\n--- Tabela Comparativa de Métricas (Modelos Base) ---")
    print(metricas.to_markdown(floatfmt=".4f"))
    logging.info(f"\nModelo Base Vencedor (F1-Score): {melhor_nome}")
    
    return melhor_modelo, melhor_nome, metricas


def otimizar_modelo(modelo_base: Pipeline, nome_modelo: str, X_train: pd.DataFrame, y_train: pd.Series, scoring: str = 'f1') -> Pipeline:
    """Otimiza o modelo vencedor utilizando busca em grade (GridSearchCV)."""
    logging.info(f"\n--- ETAPA 3: Otimização de Hiperparâmetros ({nome_modelo}) ---")

    # Hiperparâmetros a serem testados (Foco no modelo que geralmente ganha)
    grids = {
        "Random Forest": {
            'classifier__n_estimators': [100, 200, 300], # Quantidade de árvores
            'classifier__max_depth': [5, 10, 15, None],   # Profundidade máxima
            'classifier__min_samples_split': [2, 5, 10]  # Mínimo para divisão
        },
        "KNN": {
            'classifier__n_neighbors': [3, 5, 7, 9, 11],
            'classifier__weights': ['uniform', 'distance']
        },
        "Regressão Logística": {
            'classifier__C': [0.01, 0.1, 1.0, 10.0],
            'classifier__penalty': ['l2'] # Usando L2 que é o padrão com 'newton-cg', 'lbfgs', 'sag', 'saga'
        }
    }

    try:
        param_grid = grids[nome_modelo]
    except KeyError:
        logging.warning("Grid de hiperparâmetros não definido para este modelo. Retornando modelo base.")
        return modelo_base

    # GridSearchCV: Busca exaustiva com Cross-Validation (CV=5)
    search = GridSearchCV(
        modelo_base, param_grid, cv=5, scoring=scoring, n_jobs=-1, verbose=1
    )
    
    inicio = time.time()
    search.fit(X_train, y_train)
    fim = time.time()

    logging.info(f"Otimização concluída em {fim - inicio:.2f}s.")
    logging.info(f"Melhores parâmetros encontrados: {search.best_params_}")
    
    return search.best_estimator_ # Retorna o melhor Pipeline encontrado


def avaliar_final(modelo_otimizado: Pipeline, X_test: pd.DataFrame, y_test: pd.Series, nome_modelo: str, metricas_base: pd.DataFrame):
    """Compara o modelo otimizado com o base e gera o relatório final."""
    logging.info(f"\n--- ETAPA 4: Avaliação Final do Modelo Otimizado ({nome_modelo}) ---")
    
    y_pred = modelo_otimizado.predict(X_test)
    m_final = avaliar_modelo(y_test, y_pred)
    f1_base = metricas_base.loc[nome_modelo]['F1-Score']

    # Relatório de Classificação Detalhado
    print("\nRelatório de Classificação (Precision, Recall, F1-Score por classe):")
    print(classification_report(y_test, y_pred, digits=4))
    
    # Análise de Melhoria
    melhoria = (m_final['F1-Score'] - f1_base) / f1_base * 100 if f1_base != 0 else 0
    
    print("\n--- Sumário de Métricas Finais ---")
    print(f"F1-Score Base: {f1_base:.4f}")
    print(f"F1-Score Otimizado: {m_final['F1-Score']:.4f}")
    print(f"Melhoria no F1-Score: {melhoria:.2f}%")

    # Visualização da Matriz de Confusão Final
    plotar_matriz_confusao(y_test, y_pred, f'{nome_modelo} Otimizado', cmap='Greens')
    
    logging.info("Projeto concluído. Resultados disponíveis no Notebook.")


# ==============================================================================
# MAIN 
# ==============================================================================

if __name__ == '__main__':
    logging.info("Iniciando Pipeline Completo de Predição de Doenças Cardíacas (CP5)")
    
    
    
    # 1. Pré-processamento
    preprocessor, X_train, X_test, y_train, y_test = preparar_dados(
        URL_DATASET, COLUMN_NAMES, TEST_SIZE, RANDOM_STATE
    )
    
    # 2. Treinamento e Escolha do Melhor Modelo Base
    best_pipe, best_name, base_metrics = treinar_modelos_base(
        preprocessor, X_train, X_test, y_train, y_test
    )
    
    # 3. Otimização 
    optimized_pipe = otimizar_modelo(
        best_pipe, best_name, X_train, y_train
    )
    
    # Avaliação Final e Conclusão
    avaliar_final(
        optimized_pipe, X_test, y_test, best_name, base_metrics
    )